In [36]:
import os
import io
import sys
import re
import time
import math
import pickle

import numpy as np
import pandas as pd

import os
import pickle
import numpy as np
from numpy import nan as Nan
import pandas as pd

from numpy import nan as Nan
from numpy import inf as inf
from tqdm import notebook as tqdm
from scipy.sparse import csr_matrix

from data_management import read_csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
import pickle
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.io import ase
from pymatgen.transformations.standard_transformations import OxidationStateDecorationTransformation
from pymatgen.transformations.standard_transformations import AutoOxiStateDecorationTransformation
import os



In [37]:
def load_csv(path):
    save_path = os.path.join(os.getcwd(), path)
    open_file = open(save_path, 'rb')
    labels_df = pickle.load(open_file)
    open_file.close()
    return labels_df


In [38]:
experimental_data = load_csv('groups_and_oxi_states_starting_structure/df.pkl')

In [39]:
df_fall = load_csv('semi-supervised_supporting_files/labeled_data_BVSE.pkl')
df_fall = df_fall[~df_fall['conductivity'].isna()]
df_fall['composition'] = df_fall['composition'].apply(lambda x: x.replace(" ", ""))
df_fall.MP_ID.fillna(value='', inplace=True)
df_fall['MP_ID'] = df_fall['MP_ID'].apply(lambda x: x.replace("mp-", ""))
labels = set(df_fall['composition'].to_list()) & set(experimental_data['label'].to_list())

In [42]:
df = df_fall.merge(experimental_data, how='inner', left_on='composition', right_on='label')[['label', 'composition', 'ICSD_ID', 'MP_ID', 'src_id', 'conductivity']]
df[(df['ICSD_ID'] == df['src_id']) | (df['src_id'] == df['MP_ID'])]

,label,composition,ICSD_ID,MP_ID,src_id,conductivity
1,Na2Li4B2P4O16,Na2Li4B2P4O16,291512,1020646,291512,0.0
3,Li12Zn2O8,Li12Zn2O8,62137,14495,62137,0.0
7,Li18Mg6P8O32F6,Li18Mg6P8O32F6,426103,1203004,426103,0.0
8,Li4Ga4Br16,Li4Ga4Br16,61337,28326,61337,0.000007


In [51]:
labels_in_table = df['label'].to_list()

['Li16Zn4P8O32',
 'Na2Li4B2P4O16',
 'Li4B4O8',
 'Li12Zn2O8',
 'Li20Ge2P4S24',
 'Li20Ge2P4S24',
 'Li20Ge2P4S24',
 'Li18Mg6P8O32F6',
 'Li4Ga4Br16',
 'Li56P8N32',
 'Li2Sn4P6O24',
 'Li8P4O14',
 'Li24Al8N16']

In [52]:
df_ = df_fall[~df_fall['composition'].isin(labels_in_table)]

In [53]:
df_[['composition', 'conductivity', 'ICSD_ID', 'MP_ID']]

,composition,conductivity,ICSD_ID,MP_ID
17498,Li28Nd10Si22N38O14F4,0.0,262923,
17508,Li12P4S16,0.0,35018,
17575,Li1La1Nb2O7,0.0,72566,
17658,Li2P2S6,0.0,253894,
17688,Li3N1,0.0012,26540,
...,...,...,...,...
25119,Li4La4O8,0.0,239278,1020057
25132,Li3N1,0.0003,156894,2251
25138,Li12B4N8,0.0,155126,5914
25141,Li1Sm9Si6O26,0.0,83279,


In [54]:
icsds = df_['ICSD_ID'].to_list()

In [14]:
APIKEY = "gb6NLenjqf2uOU7MKjuudoEVbDQPKIKh"
with MPRester(APIKEY) as r:
    docs = r.materials.summary.search(elements=["Li"], fields=["material_id", "structure", "database_IDs"])

/usr/local/lib/python3.8/site-packages/mp_api/client/mprester.py:193: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving SummaryDoc documents: 100%|██████████| 21686/21686 [01:30<00:00, 238.89it/s]


In [57]:
l = []
doc = docs[0].structure
for i in range(len(df_)):
    l.append(doc)
df_['structure'] = l


/tmp/ipykernel_25636/2211972762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['structure'] = l


In [58]:
df_['is_in_matminer'] = False

/tmp/ipykernel_25636/1439343119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['is_in_matminer'] = False


In [59]:
df_.set_index(np.arange(len(df_)), inplace=True)

In [60]:
summ = 0
finded_icsds = []
for doc in docs:
    if 'icsd' not in doc.database_IDs.keys():
        continue
    new_list = list(map(lambda x: x.replace("icsd-", ""), doc.database_IDs['icsd']))
    for i, el in enumerate(new_list):
        if float(el) in icsds:
            for index, row in df_.iterrows():
                if row['ICSD_ID'] == int(el):
                    df_['structure'].iloc[index] = doc.structure
                    df_['is_in_matminer'].iloc[index] = True
                    summ += 1
                    break
print(summ)

/tmp/ipykernel_25636/11369577.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['structure'].iloc[index] = doc.structure
/tmp/ipykernel_25636/11369577.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['is_in_matminer'].iloc[index] = True


70


In [61]:
df_ = df_[df_['is_in_matminer'] == True]
df_[['composition', 'ICSD_ID', 'MP_ID', 'conductivity', 'structure']]

,composition,ICSD_ID,MP_ID,conductivity,structure
5,Li12Zr4O14,73835,5418,0.0,"[[-0.85825271 5.63904289 5.85194445] Li, [-0..."
12,Li18Al6F36,85171,15254,0.0,"[[4.73361578 5.83973294 7.86019592] Li, [7.360..."
13,Li8Te4O16,1485,13843,0.0,"[[4.49567795 4.49567795 7.22131246] Li, [1.530..."
15,Li64Si32N64,420126,684024,0.0,"[[3.95075135 8.37199368 9.30733141] Li, [ 2.88..."
19,Ba4Li8P8O28,280927,6175,0.0,"[[-0.31560283 1.40625965 5.29856841] Ba, [-0..."
...,...,...,...,...,...
198,Li8V8P12O48,240269,26957,0.0,"[[1.94030835 6.77215733 2.22315602] Li, [2.420..."
200,Na6Li6S6O24,14364,6783,0.0,"[[ 5.52374029 -3.53673591 4.88220423] Na, [ 4..."
202,Li4La4O8,239278,1020057,0.0,"[[3.98597944 2.7942188 3.78374601] Li, [0.866..."
203,Li3N1,156894,2251,0.0003,"[[0. 0. 1.9273193] Li, [-2.14153..."


In [62]:
save_path = os.path.join(os.getcwd(), 'mpr_test/testing_structures.pkl')
save_file = open(save_path, 'wb')
pickle.dump(df_, save_file)
save_file.close()